# Main Code

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from heapq import nlargest
from sklearn.metrics import accuracy_score

from tqdm import tqdm

import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer as optim   
from torch.utils.data import Dataset, DataLoader
from torch.nn import MSELoss

In [2]:
import import_ipynb
from network.FullyConnected_Network import FCnet
from network.Transformer_Network import ATnet
from network.Convolutional_Network import RNnet

importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/FullyConnected_Network.ipynb
importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/Transformer_Network.ipynb
importing Jupyter notebook from /mnt/batch/tasks/shared/LS_root/mounts/clusters/rhs/code/Users/qw07020821/NN_DL/nonogram_DL/network/Convolutional_Network.ipynb


## Get Data

In [3]:
print('loading...', flush = True)
X_train = np.load('./data/X_train.npy', allow_pickle= True)  # X_train and X_test is pickled
Y_train = np.load('./data/Y_train.npy')
X_test = np.load('./data/X_test.npy', allow_pickle= True)
Y_test = np.load('./data/Y_test.npy')
print('done')

loading...
done


## Preprocessing - converting conditions into 'possibility maps'

In [4]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((600000, 2, 10), (600000, 10, 10), (50000, 2, 10), (50000, 10, 10))

In [5]:
def searchcombinationsUtil(k, n):
    """
    k: number of elements (>= 1)
    n: total sum of elements
    return all possible combinations of k numbers that add up to n, regarding its order
    """
    # Recursive function
    
    if k == 1:
        return [[n]]
    else:
        output = []
        for i in range(0, n+1):
            output += [[i]+items for items in searchcombinationsUtil(k-1, n-i)]
        return output        

In [6]:
def pixel_val_calculator(constraint, N):
    """
    constraint: the condition(constraint) of a single row/column in a nonogram
    N: the length of the width/height of the nonogram
    returns a vector with length N, and each value of the vector is the possibility of the corresponding pixel to be colored
    """
    total_colored = np.sum(constraint)

    if(len(constraint) == 0):
        return [0 for _ in range(N)]
    else:
        combinations = searchcombinationsUtil(k=int(len(constraint)+1), n= int(N-total_colored-len(constraint)+1))
        output = []

        for each_combination in combinations:
            pixel_val = []
            for idx, elements in enumerate(each_combination):
                if (idx == 0) or (idx == len(constraint)):
                    pixel_val += [0 for _ in range(elements)]
                else:
                    pixel_val += [0 for _ in range(elements+1)]
                if idx != (len(constraint)):
                    pixel_val += [1 for _ in range(constraint[idx])]
            output.append(pixel_val)
        
        output = np.array(output, dtype = np.float64)
        output = np.sum(output, axis = 0)
        output/= len(combinations)

        return output

In [7]:
pixel_val_calculator([7, 1], 10)

array([0.66666667, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.33333333, 0.33333333, 0.66666667])

In [8]:
def possibility_map_generator(X):
    """
    X: (batch_size, 2, N) or (batch_size, 2, N, t) only if all constraints are composed of same number of blocks (=t)
    (2, N): each nonogram puzzle
    2 stands for each condition (row condition, column condition)
    N stands for the number of pixels (Number of total constraints)
    returns possibility_map: (batch_size, 2, N, N)
    """
    assert X.ndim in [3, 4]
    
    if X.ndim == 3:
        N = X.shape[-1]
    if X.ndim == 4:
        N = X.shape[-2]

    possibility_map = []
    for puzzles in tqdm(X):
        row_condition = puzzles[0]
        row_map = []
        for constraints in row_condition:
            row_map.append(pixel_val_calculator(constraint=constraints, N=N))
        row_map = np.array(row_map)

        column_condition = puzzles[1]
        column_map = []
        for constraints in column_condition:
            column_map.append(pixel_val_calculator(constraint=constraints, N=N))
        column_map = np.array(column_map)
        column_map = column_map.T

        possibility_map.append(np.array([row_map, column_map]))
        
    possibility_map = np.asarray(possibility_map)
    return possibility_map

In [9]:
a = np.array([[[2], [1]], [[1], [2]]])
print(a.shape)
possibility_map_generator(np.expand_dims(a,0))

100%|██████████| 1/1 [00:00<00:00, 3452.10it/s]


(2, 2, 1)


array([[[[1. , 1. ],
         [0.5, 0.5]],

        [[0.5, 1. ],
         [0.5, 1. ]]]])

## Neural Network

In [10]:
class NN_Dataset(Dataset):

    def __init__(self, X, X_mapped=None, y=None):

        assert X.ndim in [3, 4]

        if X.ndim == 3:
            N = X.shape[-1]
            self.X = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X])
        if X.ndim == 4:
            N = X.shape[-2]


        # pre-calculated possibility map if possible
        if X_mapped is not None:
            self.X_mapped = X_mapped
        else:
            self.X_mapped = possibility_map_generator(X)

        self.y = y
        
    def __getitem__(self, idx):
        inputs = np.array(self.X[idx])
        mapped_inputs = np.array(self.X_mapped[idx], dtype = np.float64)
        

        if self.y is not None:    # Train
            labels = self.y[idx]
            labels = np.array(labels, dtype=np.float64)
            return inputs, mapped_inputs, labels

        else:                     # Test
            return inputs, mapped_inputs
    
    def __len__(self):
        return len(self.X)


In [11]:
def get_loader(batch_size, shuffle, num_workers, X, X_mapped=None, y=None):
    dataset = NN_Dataset(X, X_mapped, y)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                            num_workers=num_workers)

    print(f'length : {len(dataset)}')
    return data_loader

## Training

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
def train(epochs, model, optimizer, train_loader, valid_loader, device):
    start = time.time()

    best_acc = 0
    for epoch in range(1, epochs+1):

        # Train
        model.train()    
        train_accuracies = []
        train_losses = []
        for i, (inputs, mapped_inputs, labels) in enumerate(tqdm(train_loader)):
            
            optimizer.zero_grad()
            mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
            labels = torch.tensor(labels, device=device, dtype=torch.float32)
            
            preds = model(mapped_inputs)

            loss = MSELoss()
            l = loss(preds, labels)
            
            l.backward()
            optimizer.step()

            preds = torch.flatten(preds, start_dim = 1).detach().cpu().numpy()
            labels = torch.flatten(labels, start_dim = 1).detach().cpu().numpy()


            for idx, predictions in enumerate(preds):
                total_colored = np.sum(np.array(inputs[idx][0]))
                threshold = np.amin(nlargest(total_colored, predictions))
                predictions = [1 if a >= threshold else 0 for a in predictions]
                acc = accuracy_score(predictions, labels[idx])
                train_accuracies.append(acc)
            
            
            train_losses.append(l.item())
        

        # Validation
        model.eval()
        val_accuracies = []
        val_losses = []
        with torch.no_grad():
            for inputs, mapped_inputs, labels in valid_loader:
                
                mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
                labels = torch.tensor(labels, device=device, dtype=torch.float32)

                
                preds = model(mapped_inputs)


                loss = MSELoss()
                l = loss(preds, labels)
                
                val_losses.append(l.item())

                preds = torch.flatten(preds, start_dim = 1).detach().cpu().numpy()
                labels = torch.flatten(labels, start_dim = 1).detach().cpu().numpy()


                for idx, predictions in enumerate(preds):
                    total_colored = np.sum(np.array(inputs[idx][0]))
                    threshold = np.amin(nlargest(total_colored, predictions))
                    predictions = [1 if a >= threshold else 0 for a in predictions]
                    acc = accuracy_score(predictions, labels[idx])
                    val_accuracies.append(acc)
            
            val_losses.append(l.item())

            
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        train_acc = np.mean(train_accuracies)
        val_acc = np.mean(val_accuracies)

        if best_acc < val_acc:
            best_acc = val_acc
            best_epoch = epoch
            print('saving model...', flush = True)
            if isinstance(model, ATnet):
                torch.save(model.state_dict(), './weights/ATnet.pth')
            if isinstance(model, FCnet):
                torch.save(model.state_dict(), './weights/FCnet.pth')
            if isinstance(model, RNnet):
                torch.save(model.state_dict(), './weights/RNnet.pth')
            print('done')

        print(f'Epoch:{epoch}  Train Loss:{train_loss:.3f} | Valid Loss:{val_loss:.3f}')
        print(f'Train  Acc:{train_acc:.3f}')
        print(f'Valid  Acc:{val_acc:.3f}')

    end = time.time()
    print(f'\nEpoch Process Time: {(end-start)/60:.2f}Minute')
    print(f'Best Epoch:{best_epoch}, Best Acc:{best_acc:.3f}')

In [13]:
X_train_mapped = possibility_map_generator(X_train)
X_test_mapped = possibility_map_generator(X_test)

X_train, X_val, X_train_mapped, X_val_mapped, Y_train, Y_val = train_test_split(X_train, X_train_mapped, Y_train, test_size = 0.1, random_state = 42) 

100%|██████████| 50000/50000 [03:04<00:00, 270.97it/s]


In [19]:
print(X_train.shape, X_val.shape, X_train_mapped.shape, X_val_mapped.shape, Y_train.shape, Y_val.shape)

(540000, 2, 10) (60000, 2, 10) (540000, 2, 10, 10) (60000, 2, 10, 10) (540000, 10, 10) (60000, 10, 10)


In [29]:
train_loader = get_loader(batch_size=128, shuffle=True, num_workers=6, X = X_train, X_mapped=X_train_mapped, y = Y_train) 
valid_loader = get_loader(batch_size=128, shuffle=False, num_workers=6, X = X_val, X_mapped = X_val_mapped, y = Y_val) 
epochs = 50
lr = 0.0005

num_pixels = X_train.shape[-2] if X_train.ndim is 4 else X_train.shape[-1]

# model = RNnet(n_channels = 2).to(device)

ATnet_Config = {
    'N': 10,
    'emb_dim': 64,
    'depth': 8, 
    'd_model': 64, 
    'num_heads': 8, 
    'expansion': 4, 
    'dropout': 0.1
}

model = ATnet(**ATnet_Config).to(device)
#model = FCnet(num_pixels=10, model_depth=6).to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)

length : 540000
length : 60000


In [30]:
print(model)

FCnet(
  (block1): Linear_block(
    (block): Sequential(
      (0): Linear(in_features=200, out_features=400, bias=True)
      (1): SELU()
      (2): Dropout(p=0.2, inplace=False)
    )
  )
  (block2): Sequential(
    (0): Linear_block(
      (block): Sequential(
        (0): Linear(in_features=400, out_features=400, bias=True)
        (1): SELU()
        (2): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Linear_block(
      (block): Sequential(
        (0): Linear(in_features=400, out_features=400, bias=True)
        (1): SELU()
        (2): Dropout(p=0.1, inplace=False)
      )
    )
    (2): Linear_block(
      (block): Sequential(
        (0): Linear(in_features=400, out_features=400, bias=True)
        (1): SELU()
        (2): Dropout(p=0.1, inplace=False)
      )
    )
    (3): Linear_block(
      (block): Sequential(
        (0): Linear(in_features=400, out_features=400, bias=True)
        (1): SELU()
        (2): Dropout(p=0.1, inplace=False)
      )
    )
    (4): Line

In [31]:
best_epoch, best_auprc, best_auc = train(epochs = epochs, model = model, optimizer = optimizer, train_loader = train_loader, \
                                         valid_loader = valid_loader, device = device)

  0%|          | 0/4219 [00:00<?, ?it/s]/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
100%|██████████| 4219/4219 [03:35<00:00, 19.57it/s]
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/anaconda/envs/azureml_py36/li

saving model...
done
Epoch:1  Train Loss:0.190 | Valid Loss:0.190
Train  Acc:0.636
Valid  Acc:0.627
saving model...
done
Epoch:2  Train Loss:0.191 | Valid Loss:0.190
Train  Acc:0.627
Valid  Acc:0.627
Epoch:3  Train Loss:0.563 | Valid Loss:0.575
Train  Acc:0.428
Valid  Acc:0.425
Epoch:4  Train Loss:0.565 | Valid Loss:0.560
Train  Acc:0.435
Valid  Acc:0.440
Epoch:5  Train Loss:0.517 | Valid Loss:0.485
Train  Acc:0.483
Valid  Acc:0.515
Epoch:6  Train Loss:0.451 | Valid Loss:0.435
Train  Acc:0.549
Valid  Acc:0.565
Epoch:7  Train Loss:0.423 | Valid Loss:0.415
Train  Acc:0.577
Valid  Acc:0.585
Epoch:8  Train Loss:0.413 | Valid Loss:0.410
Train  Acc:0.587
Valid  Acc:0.590
Epoch:9  Train Loss:0.411 | Valid Loss:0.410
Train  Acc:0.587
Valid  Acc:0.590
Epoch:10  Train Loss:0.415 | Valid Loss:0.420
Train  Acc:0.585
Valid  Acc:0.580
Epoch:11  Train Loss:0.421 | Valid Loss:0.410
Train  Acc:0.579
Valid  Acc:0.590
Epoch:12  Train Loss:0.405 | Valid Loss:0.410
Train  Acc:0.578
Valid  Acc:0.590


KeyboardInterrupt: 

## Inference

In [14]:
def inference(model, test_loader, device):
    pred_list = []

    model.eval()
    with torch.no_grad():
        for inputs, mapped_inputs in test_loader:
            
            mapped_inputs = torch.tensor(mapped_inputs, device=device, dtype=torch.float32)
            
            N = mapped_inputs.shape[-1]
            preds = model(mapped_inputs)
            preds = torch.flatten(preds, start_dim = 1).detach().cpu().numpy()
            
            for idx, predictions in enumerate(preds):
                total_colored = np.sum(np.array(inputs[idx][0]))
                threshold = np.amin(nlargest(total_colored, predictions))
                predictions = [1 if a >= threshold else 0 for a in predictions]
                pred_list.append(np.array(predictions).reshape(N, N))

        pred_list = np.array(pred_list)
        
    return pred_list

In [16]:
num_pixels = X_test.shape[-2] if X_test.ndim is 4 else X_test.shape[-1]


ATnet_Config = {
    'N': 10,
    'emb_dim': 64,
    'depth': 8, 
    'd_model': 64, 
    'num_heads': 8, 
    'expansion': 4, 
    'dropout': 0.1
}
model = ATnet(**ATnet_Config)
model.load_state_dict(torch.load('./weights/ATnet.pth'))
model.to(device)

test_loader = get_loader(batch_size=256, shuffle=False, num_workers=6, X = X_test, X_mapped = X_test_mapped, y = None) 

pred_list = inference(model = model, test_loader = test_loader, device = device)



length : 50000


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [31]:
X_test = np.load('./data/X_test.npy', allow_pickle=True)

In [40]:
i = 1480

condition = X_test[i]
print('row condition: ', end = "\n")
#print(*list(condition[0]))
print(*list(condition[0]), sep = '\n')
print('column condition: ', end = "\n")
#print(*list(condition[1]))
print(*list(condition[1]), sep = '\n')

print(np.round(X_test_mapped[i], 2))
print(pred_list[i])
print(Y_test[i])

row condition: 
[2, 3]
[1]
[2, 1]
[1, 1]
[]
[1]
[1, 1]
[2, 1]
[1, 1]
[1, 1]
column condition: 
[1]
[1]
[2]
[]
[2, 1, 2]
[1, 1, 3, 1]
[1, 2]
[2]
[1]
[]
[[[0.33 0.6  0.47 0.4  0.4  0.47 0.6  0.8  0.6  0.33]
  [0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 ]
  [0.25 0.46 0.39 0.36 0.32 0.29 0.25 0.21 0.21 0.25]
  [0.22 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.22]
  [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
  [0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 ]
  [0.22 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.22]
  [0.25 0.46 0.39 0.36 0.32 0.29 0.25 0.21 0.21 0.25]
  [0.22 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.22]
  [0.22 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.19 0.22]]

 [[0.1  0.1  0.11 0.   0.5  0.8  0.25 0.11 0.1  0.  ]
  [0.1  0.1  0.22 0.   0.8  0.2  0.21 0.22 0.1  0.  ]
  [0.1  0.1  0.22 0.   0.45 0.6  0.21 0.22 0.1  0.  ]
  [0.1  0.1  0.22 0.   0.4  0.4  0.25 0.22 0.1  0.  ]
  [0.1  0.1  0.22 0.   0.35 0.4  0.29 0.22 0.1  0.  ]
  [0.1  0.1  0.22 0.   0.35 1.   0.32

In [41]:
np.save('./results/pred_ATnet.npy', pred_list)
np.save('./results/label.npy', Y_test)

In [20]:
pred_list_flattened = pred_list.reshape(len(X_test), -1)
label_list_flattened = Y_test.reshape(len(X_test), -1)

test_accuracies = []

if X_test.ndim == 3:
    N = X_test.shape[-1]
    X_test = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X_test])
if X_test.ndim == 4:
    N = X_test.shape[-2]

for idx, predictions in enumerate(pred_list_flattened):  
    total_colored = np.sum(np.array(X_test[idx][0]))
    threshold = np.amin(nlargest(total_colored, predictions))
    predictions = [1 if a >= threshold else 0 for a in predictions]
    acc = accuracy_score(predictions, label_list_flattened[idx])
    test_accuracies.append(acc)

print(np.mean(test_accuracies))

0.8317584


In [39]:
correct_idx = [i for i in range(len(test_accuracies)) if test_accuracies[i] == 1.0]
print(correct_idx)

[1480, 6546, 14808, 15657, 15841, 17159, 22091, 26231, 31241, 36101, 37176, 40695, 41756, 42536, 43598, 44140, 44476, 44604, 44621, 47706]


In [22]:
i = 259

print(pred_list[i])
print(Y_test[i])
print(test_accuracies[i])

[[0 0 0 0 0 1 1 0 0 0]
 [1 0 1 1 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [1 1 1 1 0 1 1 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[0 1 0 0 0 1 1 0 0 0]
 [1 0 1 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 1 1 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]]
0.88


In [44]:
X_test_little = np.load('./data/X_test_little.npy', allow_pickle=True)
Y_test_little = np.load('./data/Y_test_little.npy')

X_test_little_mapped = possibility_map_generator(X_test_little)

test_loader_little = get_loader(batch_size=256, shuffle=False, num_workers=6, X = X_test_little, X_mapped = X_test_little_mapped, y = None) 

pred_list_little = inference(model = model, test_loader = test_loader_little, device = device)


pred_list_little_flattened = pred_list_little.reshape(len(X_test_little), -1)
label_list_little_flattened = Y_test_little.reshape(len(X_test_little), -1)

test_accuracies_little = []

if X_test_little.ndim == 3:
    N = X_test_little.shape[-1]
    X_test_little = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X_test_little])
if X_test.ndim == 4:
    N = X_test_little.shape[-2]

for idx, predictions in enumerate(pred_list_little_flattened):  
    total_colored = np.sum(np.array(X_test_little[idx][0]))
    threshold = np.amin(nlargest(total_colored, predictions))
    predictions = [1 if a >= threshold else 0 for a in predictions]
    acc = accuracy_score(predictions, label_list_little_flattened[idx])
    test_accuracies_little.append(acc)

print(np.mean(test_accuracies_little))

100%|██████████| 40000/40000 [00:43<00:00, 912.56it/s] 
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


length : 40000
0.9416190000000001


In [55]:
np.save('./results/pred_ATnet_little.npy', pred_list_little)
np.save('./results/label_little.npy', Y_test_little)

In [48]:
X_test_many = np.load('./data/X_test_many.npy', allow_pickle=True)
Y_test_many = np.load('./data/Y_test_many.npy')

X_test_many_mapped = possibility_map_generator(X_test_many)

test_loader_many = get_loader(batch_size=256, shuffle=False, num_workers=6, X = X_test_many, X_mapped = X_test_many_mapped, y = None) 

pred_list_many = inference(model = model, test_loader = test_loader_many, device = device)


pred_list_many_flattened = pred_list_many.reshape(len(X_test_many), -1)
label_list_many_flattened = Y_test_many.reshape(len(X_test_many), -1)

test_accuracies_little = []

if X_test_many.ndim == 3:
    N = X_test_many.shape[-1]
    X_test_many = np.array([[np.array([k + [0]*(N - len(k))for k in j]) for j in i] for i in X_test_many])
if X_test.ndim == 4:
    N = X_test_many.shape[-2]

for idx, predictions in enumerate(pred_list_many_flattened):  
    total_colored = np.sum(np.array(X_test_many[idx][0]))
    threshold = np.amin(nlargest(total_colored, predictions))
    predictions = [1 if a >= threshold else 0 for a in predictions]
    acc = accuracy_score(predictions, label_list_many_flattened[idx])
    test_accuracies_little.append(acc)

print(np.mean(test_accuracies_little))

100%|██████████| 40000/40000 [01:20<00:00, 496.99it/s]
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


length : 40000
0.992267


In [54]:
X_test_many = np.load('./data/X_test_many.npy', allow_pickle=True)

i = 101

condition = X_test_many[i]
print('row condition: ', end = "\n")
#print(*list(condition[0]))
print(*list(condition[0]), sep = '\n')
print('column condition: ', end = "\n")
#print(*list(condition[1]))
print(*list(condition[1]), sep = '\n')

print(np.round(X_test_many_mapped[i], 2))

print(pred_list_many[i])
print(Y_test_many[i])

row condition: 
[1, 2, 1]
[1, 5, 1]
[1, 2, 2]
[4, 1, 1]
[1, 3, 4]
[1, 3, 1, 1]
[1, 6, 1]
[1, 1, 1, 1]
[2, 1, 1, 1]
[2, 4, 1]
column condition: 
[8]
[2, 1, 2]
[2, 1]
[2, 6]
[3, 3, 1]
[3, 2, 2]
[2, 1, 2, 1]
[1, 3, 1]
[1, 1, 1]
[7, 1]
[[[0.43 0.29 0.31 0.46 0.51 0.51 0.46 0.31 0.29 0.43]
  [0.75 0.25 0.5  1.   1.   1.   1.   0.5  0.25 0.75]
  [0.5  0.3  0.35 0.55 0.6  0.55 0.4  0.45 0.8  0.5 ]
  [0.6  0.9  1.   1.   0.4  0.4  0.4  0.4  0.3  0.6 ]
  [1.   0.   1.   1.   1.   0.   1.   1.   1.   1.  ]
  [0.8  0.2  0.6  1.   1.   0.4  0.4  0.6  0.2  0.8 ]
  [1.   0.   1.   1.   1.   1.   1.   1.   0.   1.  ]
  [0.57 0.29 0.4  0.37 0.37 0.37 0.37 0.4  0.29 0.57]
  [0.67 0.93 0.33 0.47 0.4  0.4  0.4  0.47 0.27 0.67]
  [0.75 1.   0.25 0.5  1.   1.   1.   0.5  0.25 0.75]]

 [[0.33 0.5  0.25 0.67 0.75 0.75 0.8  0.5  0.38 0.67]
  [0.67 0.8  0.46 1.   1.   1.   1.   0.3  0.27 1.  ]
  [1.   0.45 0.39 0.33 1.   1.   0.2  0.35 0.29 1.  ]
  [1.   0.4  0.36 0.33 0.25 0.25 0.6  0.55 0.29 1.  ]
  [1.   0.

In [56]:
np.save('./results/pred_ATnet_many.npy', pred_list_many)
np.save('./results/label_many.npy', Y_test_many)